# Computer vision operation

> Different computer vision operations will be implemented here

In [ ]:
#| default_exp cv_ops

In [2]:
#| export
import pandas as pd
from pathlib import Path
import numpy as np
from tqdm import tqdm
from fastcore.all import *
import matplotlib as mpl
import matplotlib.pyplot as plt
from fastcore.all import *
import shutil
import cv2
from typing import Union, List, Tuple, Dict
import pandas as pd
from skimage import io, morphology, measure
from skimage.filters import threshold_multiotsu
from scipy.ndimage import (label, sum, binary_fill_holes)
import os
from tqdm.auto import tqdm
import argparse

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| hide
data_path=Path(r'/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images')
data_path.ls()

(#1642) [Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_162_p_9.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_11_p_9.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_70_p_5.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_67_p_1.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_74_p_9.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_16_p_5.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_23_p_1.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_112_p_6.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_121_p_11.png'),Path('/home/hasan/Schreibtisch/projects/data/microscopy/patch_train_images/img_152_p_1.png')...]

In [5]:
#| hide
from cv_tools.core import *
import matplotlib.pyplot as plt

In [6]:
#| export
def multi_otsu(img:np.array, classes:int=3)->np.array:
    """Multi-Otsu thresholding-> returns image of type np.uint8"""
    thresholds = threshold_multiotsu(img, classes)
    regions = np.digitize(img, bins=thresholds)
    # Scale the regions to span full 8-bit range (0-255) by multiplying by 255/(num_classes-1)
    # E.g. for 3 classes, maps [0,1,2] to [0,127,255]
    regions = regions*(255//(classes-1))
    return regions.astype(np.uint8)

In [ ]:
#| export
def remove_small_objects(
        binary_mask:np.ndarray,  # np.ndarry
        size_threshold:Union[float, int] # 
        ):
    'From a binary mask remove small objects'
    labeled_mask, num_labels = label(binary_mask)
    sizes = sum(binary_mask, labeled_mask, range(num_labels + 1))
    mask_sizes = sizes[labeled_mask]
    binary_mask = mask_sizes > size_threshold
    return binary_mask

In [ ]:
#| export
def fill_holes_in_objects(binary_mask):
    'Fill holes inside binary mask'
    filled_mask = binary_fill_holes(binary_mask)
    return filled_mask

In [ ]:
#| export
def convert_to_rotated_rectangles(binary_mask):
    'Convert binary mask arbitary from to  rotated rectange '
    contours, _ = cv2.findContours(binary_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    new_mask = np.zeros_like(binary_mask, dtype=np.uint8)
    for contour in contours:
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.intp(box)
        cv2.drawContours(new_mask, [box], 0, (1), -1)
    return new_mask

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()